In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

from _brush.individual import RegressorIndividual
from _brush import SearchSpace, Parameters, Dataset

from pybrush import DeapRegressor

ImportError: libcbrush.so: cannot open shared object file: No such file or directory

In [ ]:
# Load the diabetes dataset
diabetes = datasets.load_diabetes()

# Use only one feature
X = diabetes.data[:, None, 2]
y = diabetes.target

import pandas as pd

# df = pd.read_csv("https://raw.githubusercontent.com/gAldeia/hashing-symbolic-expressions/master/data/lexicase_paper/d_airfoil.txt?token=GHSAT0AAAAAACPJ5UIOJY42GOUHC4GKZOBOZPS7BHA")
# X = df.drop('label', axis=1)
# y = df['label']

print(X.shape, y.shape)
# Split the data into training/testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from pybrush import engine
print("imported")

# Validation is to hold some part of the data as the inner validation split
dataset = Dataset(X=X_train, y=y_train, validation_size=0.75)
print("dataset")

params = Parameters()
print("parameters")

brush_estimator = engine.RegressorEngine(params)
print("estimator")

print(brush_estimator.params.pop_size)
brush_estimator.params.pop_size = 100
brush_estimator.params.gens = 100
brush_estimator.params.num_islands = 5
brush_estimator.params.max_size = 2**6
brush_estimator.params.max_depth = 6
brush_estimator.params.n_jobs = 5
brush_estimator.params.objectives = ["error", "size"]
print(brush_estimator.params.pop_size)

print(brush_estimator.is_fitted)
print(brush_estimator.best_ind.program.get_model())


In [ ]:
reg = DeapRegressor(
    gens=100, pop_size=100, max_size=2**6, max_depth=6,
    num_islands=1,
    n_jobs=1,
    objectives=['error', 'size'], #, 'complexity'],
    verbosity=1
)

In [ ]:

print("starting to run")

brush_estimator.run(dataset)
print("done")


In [ ]:
print(brush_estimator.is_fitted)
print(brush_estimator.best_ind.program.get_model())

In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(
    brush_estimator.best_ind.program.predict(X_test), y_test, squared=False)

In [ ]:
brush_estimator.best_ind.fitness.values

In [ ]:
regr = RegressorIndividual()
print(dir(regr))

# Validation is to hold some part of the data as the inner validation split
dataset = Dataset(X=X_train, y=y_train, validation_size=0.75)
ss = SearchSpace(dataset)
params = Parameters()

regr.init(ss, params)

# regr.fit(X_train, y_train)
regr.program.get_model()

In [ ]:
reg.fit(X_train, y_train)

In [ ]:
from pybrush import BrushRegressor

reg2 = BrushRegressor(
    gens=100, pop_size=100, max_size=2**6, max_depth=6,
    num_islands=1,
    n_jobs=3,
    objectives=['error', 'size'], #, 'complexity'],
    verbosity=1
).fit(X_train, y_train)

In [ ]:
reg.best_estimator_.fitness.values

In [ ]:
from pybrush import RegressorEvaluator

# RegressorEvaluator()
print( reg.best_estimator_.program.get_model() )
print( reg.best_estimator_.fitness.values )

RegressorEvaluator().assign_fit(
    reg.best_estimator_, reg.data_, reg.parameters_, True)
print( reg.best_estimator_.fitness.values )


In [ ]:
from pybrush import RegressorEvaluator
import numpy as np
print(regr.fitness.weights)
regr.objectives = ['error', 'size']
print(regr.fitness.weights)

regr.fitness.values = [1, 2]
regr.init(reg.search_space_, reg.parameters_)
regr.program.fit(reg.data_)

print(regr.program.get_model())
print(regr.fitness.wvalues)
print(regr.fitness.values)

RegressorEvaluator().assign_fit(
    regr, reg.data_, reg.parameters_, False)
print( regr.fitness.values )

def _error(ind, data):
    MSE = np.mean( (data.y-ind.program.predict(data))**2 )
    if not np.isfinite(MSE): # numeric erros, np.nan, +-np.inf
        MSE = np.inf

    return MSE

def _fitness_validation(ind, data):
    # Fitness without fitting the expression, used with validation data

    ind_objectives = {
        "error"     : _error(ind, data),
        "size"      : ind.program.size(),
        "complexity": ind.program.complexity()
    }
    return [ ind_objectives[obj] for obj in reg.objectives ]

def _fitness_function(ind, data):
    return _fitness_validation(ind, data)

print(_fitness_function(regr, reg.data_))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.datasets import load_iris, load_breast_cancer

from pybrush import individual
from pybrush import ClassifierEvaluator

# Load the iris dataset
iris = load_breast_cancer()
X = iris.data
y = iris.target
print(np.unique(y))

clf = individual.ClassifierIndividual()
print(dir(clf))

# c=True will add logistic function into the search space
# Validation is to hold some part of the data as the inner validation split
dataset = Dataset(X=X, y=y, c=True, validation_size=0.0)
ss = SearchSpace(dataset)
params = Parameters()

clf.init(ss, params)

# clf.fit(X_train, y_train)
clf.program.get_model()

clf.objectives = ['error', 'size']
print(clf.fitness.weights)

clf.fitness.values = [1, 2]
clf.program.fit(dataset)

print(clf.program.get_model())
print(clf.fitness.wvalues)
print(clf.fitness.values)

ClassifierEvaluator().assign_fit(clf, dataset, params, False)
print( clf.fitness.values )
def _error(ind, data):
    probas = ind.program.predict_proba(data)
    print(probas[:3])
    probas = np.array([probas, 1-probas]).T
    print(probas.shape)
    print(probas[:3, :])
    ERR = log_loss(data.y, probas, labels=['a', 'b'])
    if not np.isfinite(ERR): # numeric erros, np.nan, +-np.inf
        ERR = np.inf

    return ERR

def _fitness_validation(ind, data):
    # Fitness without fitting the expression, used with validation data

    ind_objectives = {
        "error"     : _error(ind, data),
        "size"      : ind.program.size(),
        "complexity": ind.program.complexity()
    }
    return [ ind_objectives[obj] for obj in clf.objectives ]

def _fitness_function(ind, data):
    return _fitness_validation(ind, data)

print(_fitness_function(clf, dataset))

In [ ]:
from pybrush import BrushClassifier

clf = BrushClassifier(
    gens=10, pop_size=10, max_size=2**5, max_depth=5,
    num_islands=1,
    n_jobs=3,
    objectives=['error', 'size'], #, 'complexity'],
    verbosity=1,
    functions={"Add":1.0,"Logistic":1.0},
).fit(X, y)
clf.best_estimator_.program.get_model()

In [ ]:
clf_eval = ClassifierEvaluator()
clf_eval.scorer

In [ ]:
#from pybrush import RegressorSelector

from _brush import RegressorSelector

# RegressorSelector().select([reg.best_estimator_], params)